# FCNN training

In [1]:
# This is only valid when the package is not installed
import sys
sys.path.append('../../') # two folders up
import DeepINN as dp
import torch

Using default backend: PyTorch
Using Pytorch:  2.0.1+cu117


## Geometry

In [2]:
# A simple 1D geometry
X = dp.spaces.R1('x')
Line = dp.domains.Interval(X, 0, 1)

In [3]:
left_bc = dp.constraint.DirichletBC(geom = Line,
                                      function = lambda X: torch.tensor([0.0]),
                                      sampling_strategy = "grid",
                                      no_points = 1, # you can use more points. there are conditions to deal with stupid conditions. 
                                      filter_fn = lambda x: x[:] == 0.0)
right_bc = dp.constraint.DirichletBC(geom = Line,
                                      function = lambda X: torch.tensor([1.0]),
                                      sampling_strategy = "grid",
                                      no_points = 1, # you can use more points. there are conditions to deal with stupid conditions. 
                                      filter_fn = lambda x: x[:] == 1.0)
interior_points = dp.constraint.PDE(geom = Line,
                                    sampling_strategy= "grid",
                                    no_points = 20)

In [4]:
# dp.utils.scatter(X, interior_points.sampler_object(), dpi = 50)
# dp.utils.scatter(X, left_bc.sampler_object(), dpi = 50)
# dp.utils.scatter(X, right_bc.sampler_object(), dpi = 50)

## PDE

In [5]:
def laplace(X,y):
    """
    1D Laplace equation.
    u__x = 0 
    """
    dy_x = dp.constraint.Jacobian(X, y)
    dy_xx = dp.constraint.Jacobian(X, y)(i = 0, j = 0)

    return dy_xx

In [6]:
domain = dp.domain.Generic(laplace,
                            interior_points,
                            [left_bc, right_bc])

## Network

In [7]:
activation = "tanh"
initialiser = "Xavier normal"
layer_size = [1] + [5] * 1 + [1]

In [8]:
net = dp.nn.FullyConnected(layer_size, activation, initialiser)
model = dp.Model(domain, net)

In [9]:
optimiser = "adam"
lr=0.001
metrics="MSE"

model.compile(optimiser, lr, metrics, device = "cuda")

/home/hell/Desktop/repos/DeepINN/Tutorials/5. FCNN/../../DeepINN/geometry/samplers/grid_samplers.py:78: UserWarning: First iteration did not find any valid grid points, for
                             the given filter.
                             Will try again with n = 10 * self.n_points. Or
                             else use only random points!
  warnings.warn("""First iteration did not find any valid grid points, for
Domain compiled
Network compiled


In [10]:
model.optimiser_function, model.lr, model.metric

(torch.optim.adam.Adam, 0.001, MSELoss())

In [11]:
model.train(iterations = 100)

Iteration: 1 	 BC Loss: 2.1174	 PDE Loss: 12.3646 	 Loss: 14.4820
Iteration: 11 	 BC Loss: 1.9977	 PDE Loss: 11.2331 	 Loss: 13.2308
Iteration: 21 	 BC Loss: 1.8826	 PDE Loss: 10.1766 	 Loss: 12.0593
Iteration: 31 	 BC Loss: 1.7726	 PDE Loss: 9.2003 	 Loss: 10.9729
Iteration: 41 	 BC Loss: 1.6678	 PDE Loss: 8.3045 	 Loss: 9.9723
Iteration: 51 	 BC Loss: 1.5684	 PDE Loss: 7.4863 	 Loss: 9.0548
Iteration: 61 	 BC Loss: 1.4745	 PDE Loss: 6.7411 	 Loss: 8.2156
Iteration: 71 	 BC Loss: 1.3859	 PDE Loss: 6.0632 	 Loss: 7.4491
Iteration: 81 	 BC Loss: 1.3025	 PDE Loss: 5.4470 	 Loss: 6.7495
Iteration: 91 	 BC Loss: 1.2241	 PDE Loss: 4.8870 	 Loss: 6.1111
Iteration: 101 	 BC Loss: 1.1505	 PDE Loss: 4.3783 	 Loss: 5.5287
Training finished


In [12]:
# model.iter = 1
# model.train(iterations = 2000)